1) Carga de librerias necesarias

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.preprocessing import LabelEncoder


2) Carga de bases de datos

In [28]:
#Se guarda en variables los datos de los archivos csv
client_train = pd.read_csv('client_train.csv')
invoice_train = pd.read_csv('invoice_train.csv')

C:\Users\luiso\AppData\Local\Temp\ipykernel_17196\1930133614.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  invoice_train = pd.read_csv('invoice_train.csv')


3) Verificar Unicidad de client_id:

In [29]:
#En la base de datos client_train se cuenta el número de veces que se repite el id de un cliente
client_counts = client_train['client_id'].value_counts()
print(client_counts.head())

client_id
train_Client_0        1
train_Client_59347    1
train_Client_5936     1
train_Client_59359    1
train_Client_59358    1
Name: count, dtype: int64


In [30]:
#En la base de datos invoice_train se cuenta el número de veces que se repite el id de un cliente
invoice_counts = invoice_train['client_id'].value_counts()
print(invoice_counts.head())


client_id
train_Client_116878    439
train_Client_9263      380
train_Client_130245    327
train_Client_59131     317
train_Client_61728     310
Name: count, dtype: int64


4) Se contruye un historico de consumo

In [31]:
# Se convierte la variable 'invoice_date' a formato fecha 
invoice_train['invoice_date'] = pd.to_datetime(invoice_train['invoice_date'])

# Se ordena 'invoice_train' por 'client_id' y 'invoice_date'
invoice_train_sorted = invoice_train.sort_values(by=['client_id', 'invoice_date'])

# Se crea una variable que contiene el historico de facturación de cada cliente y los datos de la base de datos 'client_train'
historico_consumo = pd.merge(invoice_train_sorted, client_train, on='client_id', how='left')

5) Análisis Exploratorio de Datos (EDA)

In [32]:
# Se observa el historico de consumo
historico_consumo.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,...,consommation_level_4,old_index,new_index,months_number,counter_type,disrict,client_catg,region,creation_date,target
0,train_Client_0,2005-10-17,11,1335667,0,203,6,1,124,0,...,0,3685,3809,4,ELEC,60,11,101,31/12/1994,0.0
1,train_Client_0,2006-02-24,11,1335667,0,203,6,1,141,0,...,0,3809,3950,4,ELEC,60,11,101,31/12/1994,0.0
2,train_Client_0,2006-06-23,11,1335667,0,203,6,1,162,0,...,0,3950,4112,4,ELEC,60,11,101,31/12/1994,0.0
3,train_Client_0,2006-10-18,11,1335667,0,203,6,1,159,0,...,0,4112,4271,4,ELEC,60,11,101,31/12/1994,0.0
4,train_Client_0,2007-02-26,11,1335667,0,203,6,1,182,0,...,0,4271,4453,4,ELEC,60,11,101,31/12/1994,0.0


In [33]:
# Se describe el historico de consumo
historico_consumo.describe()

,invoice_date,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,disrict,client_catg,region,target
count,4476749,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06,4.476749e+06
mean,2013-03-18 07:42:34.444127232,2.012804e+01,1.230587e+11,1.724884e+02,7.321702e+00,1.003040e+00,4.109795e+02,1.093225e+02,2.030620e+01,5.292588e+01,1.776700e+04,1.834970e+04,4.483095e+01,6.348246e+01,1.194381e+01,2.103584e+02,7.887688e-02
min,1977-06-09 00:00:00,8.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+01,1.100000e+01,1.010000e+02,0.000000e+00
25%,2010-01-19 00:00:00,1.100000e+01,1.211080e+05,5.000000e+00,6.000000e+00,1.000000e+00,7.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.791000e+03,2.056000e+03,4.000000e+00,6.200000e+01,1.100000e+01,1.030000e+02,0.000000e+00
50%,2013-07-10 00:00:00,1.100000e+01,4.945610e+05,2.030000e+02,8.000000e+00,1.000000e+00,2.740000e+02,0.000000e+00,0.000000e+00,0.000000e+00,7.690000e+03,8.192000e+03,4.000000e+00,6.200000e+01,1.100000e+01,3.010000e+02,0.000000e+00
75%,2016-09-03 00:00:00,4.000000e+01,1.115161e+06,2.070000e+02,9.000000e+00,1.000000e+00,6.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,2.166000e+04,2.234300e+04,4.000000e+00,6.300000e+01,1.100000e+01,3.090000e+02,0.000000e+00
max,2019-12-07 00:00:00,4.500000e+01,2.798115e+13,6.000000e+02,4.130000e+02,5.000000e+01,9.999100e+05,9.990730e+05,6.449200e+04,5.479460e+05,2.800280e+06,2.870972e+06,6.366240e+05,6.900000e+01,5.100000e+01,3.990000e+02,1.000000e+00
std,NaN,1.347256e+01,1.657267e+12,1.338871e+02,1.571654e+00,3.083466e-01,7.573080e+02,1.220123e+03,1.574239e+02,8.754725e+02,4.036693e+04,4.095321e+04,3.128335e+03,3.273403e+00,6.022452e+00,1.035828e+02,2.695465e-01


In [34]:
# Se observa la distribución de las variables
historico_consumo.value_counts()

client_id            invoice_date  tarif_type  counter_number  counter_statue  counter_code  reading_remarque  counter_coefficient  consommation_level_1  consommation_level_2  consommation_level_3  consommation_level_4  old_index  new_index  months_number  counter_type  disrict  client_catg  region  creation_date  target
train_Client_66981   2006-07-17    11          132701          5               420           6                 1                    0                     0                     0                     0                     5946       5946       2              ELEC          63       51           313     20/11/1996     1.0       3
train_Client_3636    2012-01-18    40          312650          5               5             6                 1                    0                     0                     0                     0                     623        623        2              GAZ           60       11           101     21/11/1988     0.0       2
train_Client_7066    

In [35]:
# Se cuenta la variable 'target'
target_counts = historico_consumo['target'].value_counts()
print(target_counts)



target
0.0    4123637
1.0     353112
Name: count, dtype: int64


6) Preparación de datos

In [36]:
#Se eliminan las columnas que no se utilizarán
historico_consumo = historico_consumo.drop(['invoice_date', 'creation_date'], axis=1)

# Lista de columnas categóricas para convertir a formato numérico mediante One-Hot Encoding
columns_to_encode = ['tarif_type', 'counter_statue', 'counter_code', 'reading_remarque', 'client_catg', 'region', 'counter_type']

# Inicializa el codificador de etiquetas
label_encoder = LabelEncoder()

# Ajusta el codificador de etiquetas al 'client_id' y transforma 'client_id' a valores numéricos
historico_consumo['client_id_encoded'] = label_encoder.fit_transform(historico_consumo['client_id'])

# Se elimina la columna 'client_id' original
historico_consumo = historico_consumo.drop(['client_id'], axis=1)

# Aplica la codificación One-Hot a 'historico_consumo'
historico_consumo_encoded = pd.get_dummies(historico_consumo, columns=columns_to_encode)




In [37]:
# Se define la variable 'X' (independiente) y 'y' (dependiente) al dataset 'historico_consumo_encoded' (ya codificado)
X = historico_consumo_encoded.drop(['target'], axis=1) 
y = historico_consumo_encoded['target']

# Se divide la base de datos en entrenamiento y prueba (80/20 respectivamente), dentro del mismo dataset 'historico_consumo'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Nos aseguguramos de que no hayan obejtos en la columa x y que y sea numero
object_columns = X_train.select_dtypes(include=['object']).columns
print(object_columns)
print(y_train.dtype)



Index([], dtype='object')
float64


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
import sys

# Verificar si hay una GPU disponible
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
else:
    sys.exit("GPU not detected, stopping the execution.")

# Preprocesamiento: Eliminar filas con valores faltantes y asegurarse de que todo sea numérico
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Definir el modelo de red neuronal
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

model = SimpleNN(X_train.shape[1]).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reducir la tasa de aprendizaje

# Convertir los datos de entrenamiento y prueba a tensores y cargarlos en DataLoader
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Entrenamiento del modelo con monitoreo de la pérdida en el conjunto de prueba
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    
    # Evaluación después de cada época
    model.eval()
    y_pred_list = []
    y_true_list = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.squeeze().round()
            y_pred_list.extend(preds.cpu().numpy())
            y_true_list.extend(labels.cpu().numpy())
    
    # Calcular las métricas
    accuracy = accuracy_score(y_true_list, y_pred_list)
    precision = precision_score(y_true_list, y_pred_list)
    recall = recall_score(y_true_list, y_pred_list)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')

GPU detected: NVIDIA GeForce RTX 3090


c:\Users\luiso\anaconda3\envs\fraud-detection-environment\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5, Loss: 0.0000
Accuracy: 0.9211, Precision: 0.0000, Recall: 0.0000


c:\Users\luiso\anaconda3\envs\fraud-detection-environment\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/5, Loss: 8.6957
Accuracy: 0.9211, Precision: 0.0000, Recall: 0.0000


KeyboardInterrupt: 

7) Entrenamiento de un arbol de de desición

In [ ]:
# Inicializar y entrenar el Árbol de Decisión
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Realizamos el entrenamiento
accuracy = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
precision = cross_val_score(clf, X_train, y_train, cv=5, scoring='precision', n_jobs=-1)
recall = cross_val_score(clf, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)

# Se imprime el resultado del entrenamiento
print(f'Accuracy promedio: {np.mean(accuracy)}')
print(f'Precision promedio: {np.mean(precision)}')
print(f'Recall promedio: {np.mean(recall)}')

Accuracy promedio: 0.9730772805575734
Precision promedio: 0.823848291977949
Recall promedio: 0.8377383207350103
